# DL4NLP SS17 Home Exercise 05
----------------------------------
**Due until Tuesday, 30.05. at 13:00**

## Task 1 Mandatory Paper (1P)

Read [this week's mandatory paper](http://www.aclweb.org/anthology/P14-2050) and state two examples for which dependency-based embeddings are likely to produce better results than conventional neural embeddings (*herkömmliche neuronale Embeddings*).

## Task 2 Extrinsic Embedding Evaluation via Sequence Tagging (9P)

In this task, you will perform extrinsic evaluation of word embeddings on a sequence tagging task.

#### Data
We use data from the CoNLL-2003 shared task on Named Entity Recognition. The data is uploaded in moodle as [hex05_data_v2.zip](https://moodle.informatik.tu-darmstadt.de/pluginfile.php/54803/mod_assign/introattachment/0/hex05_data_v2.zip?forcedownload=1). It is pre-partioned into a training, development and test set.

The files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags are in "BIO" format. Here is an example:

```
   U.N.         NNP  B-NP  B-ORG 
   official     NN   I-NP  O 
   Ekeus        NNP  I-NP  B-PER 
   heads        VBZ  B-VP  O 
   for          IN   B-PP  O 
   Baghdad      NNP  B-NP  B-LOC 
   .            .    O     O 
```

"BIO" stands for Beginning, Inside and Outside. A "`B-`" prefix before a tag indicates the beginning of a chunk and an "`I-`" prefix before a tag indicates that the tag is inside a chunk. An "`O`" tag indicates that a token belongs to no chunk. Sometimes, this tagging scheme is also referred to as "IOB2".

#### Pre-trained Embeddings
[hex05_embeddings.zip](https://moodle.informatik.tu-darmstadt.de/pluginfile.php/54803/mod_assign/introattachment/0/hex05_embeddings.zip?forcedownload=1) contains six pre-trained word embedding files in text format:

| Filename  | Description  |
|---|---|
| `hex05.word2vec.300d.txt`  | 300-dimensional word2vec embeddings by Mikolov et al. 2013  |
| `hex05.glove.6B.50d.txt`  | 50-dimensional GloVe vectors, trained on a corpus of 6B tokens by Pennington et al. 2014  |
| `hex05.glove.840B.300d.txt`  | 300-dimensional GloVe vectors, trained on a corpus of 840B tokens by Pennington et al. 2014  |
| `hex05.levy.300d.txt`  | 300-dimensional dependency embeddings by Levy et al. 2014  |
| `hex05.komninos.300d.txt`  | 300-dimensional hybrid embeddings of word2vec and levy embeddings by Komninos et al. 2016  |
| `hex05.bilbowa.parallel.200d.txt`  | 200-dimensional vectors of the English part of bilingual BilBOWA embeddings by Gouws et al. 2014 |
The files were preprocessed so that they contain only those words which are necessary for this home exercise.

#### Procedure
To perform sequence tagging, we interpret the task as a classification of single words and their context (see the [extra material in Moodle](https://moodle.informatik.tu-darmstadt.de/mod/resource/view.php?id=7630)). The CoNLL files offer three tagging tasks, namely the prediction of POS tags, of Chunking tags and of NER tags. We will consider these tasks separately, i.e. no multi-task learning.

#### Hints on the Submission Format
* Please submit your python code for all the tasks where it is applicable. Make sure to include comments explaining complicated/non-obvious sections of your code.
* Please also submit a copy of the console output of your code execution. Your code might run in 10 minutes on your watercooled battlestation, but it might not run in 10 minutes for the person who corrects your home exercises. Thank you!
    * If your code didn't finish running, please be honest and add a short statement saying so. Thank you again!

### Task 2.1 What are we doing here, anyway? (1P)
Look up the definition of the three NLP tasks POS-tagging, Chunking and Named Entity Recognition. Quickly explain their idea/purpose in up to two sentences per task.

...

### Task 2.2 Dataset Reader (1P)
Implement a reader for the CoNLL-2003 data which returns the data in a usable format. What is "usable" is up to your preferences.

### Task 2.3 Data Windowing (1P)
Several preprocessing steps are necessary before the data can be fed to a neural network. Implement preprocessing functionality, which:
* given one of the three tagging tasks (POS, Chunking, NER), throws away the tags of the two unrelated tasks
* given window size $k$ (an uneven integer), returns a windowed version of each sentence, where each window is associated with the tag of the middle word in the window. It's easier to understand with an example:
    * **Example**: We have $k = 3$ and want to tag named entities. For the sentence "Donald Trump is the president of the USA." and named entity tags "`B-PER I-PER O O O O O B-LOC O`", your preprocessing should return something similar to:
    
| window | tag |
|---|---|
| ["\_\_PADDING\_\_", "Donald", "Trump"] | `B-PER` |
| ["Donald", "Trump", "is"] | `I-PER` |
| ["Trump", "is", "the"] | `O` |
| ... | ... |

**To get the 1P, print your preprocessed version of the first sentence from the training data for the Named Entity Recognition task and $k=3$.**

### Task 2.4 Evaluation (1P)
Implement a method which, given the sequence of predicted tags and the sequence of the true tags, returns the accuracy of the prediction.

**1P Bonus if you compute the macro F1 measure.**

### Task 2.5 MLP (4P)
Implement a multilayer perceptron with either TensorFlow or Keras (your choice).

As the input, your MLP should accept a context window of size $k$. It should transform the words into embedding vectors which are fed to several dense layers. The output layer should be softmax and the loss should be cross-entropy loss. The network output should then represent the probabilities of each tag for the middle word in the context window. Choose the tag with the highest probability as the final output.

**Hints:**
* The sentence structure is irrelevant for the MLP.
* Define an embedding vector for your "\_\_PADDING\_\_" string and another for out-of-vocabulary words. Make sure to use the same embedding vectors for training and testing.
* Since we are using cross-entropy loss, the tags (our class labels) need to be encoded as one-hot vectors. The one-hot vectors will have to be decoded to obtain tags during testing.
* If you would like to use the Embedding layer from Keras (not mandatory), then you can find a nice example [here](https://github.com/fchollet/keras/issues/853).

#### a) Results with word2vec Embeddings

Train your MLP with the following configuration, then report its accuracy(/F1) on the dev set:
* part-of-speech tagging
* 300-dimensional word2vec embeddings
* context window size $k = 7$ 
* 1 hidden layer with a dimension of 1000 and using tanh activation
* hidden layer weights initialized by **uniform Glorot initialization**
* batch size 200
* 10 epochs
* Adam optimizer (with default parameters)

Hint on the hidden layer initialization:
* TensorFlow: [tf.contrib.layers.xavier_initializer()](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/xavier_initializer)
* Keras: [glorot.uniform](https://keras.io/initializers/#glorot_uniform)
    * this is already the default setting for `Dense` layers in Keras

#### b) Komninos Embeddings
Retrain your MLP with Komninos embeddings (the other hyperparameters stay the same). Do the results on the dev set differ from task a)?

### Task 2.6 Experiments (1P)
Experiment with a different tagging task and different embeddings/hyperparameters.

To get the 1P:
* Run your MLP for the Chunking or Named Entity Recognition task. Report the accuracy/F1 on the test set.
* Optimize the parameters of your MLP for the POS-tagging task. Try out at least three different context window sizes and three different pre-trained embeddings. Report the accuracy/F1 of the best parameter set on the test set.

More experimentation (elaborate hyperparameter optimization, different network architecture, different activation functions, etc., etc.) is welcome but not mandatory. We'd love to hear about your best result!